In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import sys

## Goal: define flash drougths

In this notebook, I am working to a script that defines the flash drought from an indexes and a number of criteria. 


### The criteria we need (input): 

- we choose an index **index** of a certain time scale **scale**
- we compute the first order derivate of the index with a rolling-mean window **window** 
- the starting index value has to be higher than **start_threshold**
- the end index value has to be lower than **end_threshold**
- the first order derivative of the index must experience a jump in value of minimal size **jump**


### Output of the script:

- CSV file with start dates of flash droughts (3 columns: year, month, day)
- file name must contain the values of the above input criteria

### steps: 
 1. read in Index netcdf
 2. calculate first order derivative
 3. find where data fits the following conditions:
    1. index(t) > start_threshold
    2. index(t+window) < end_threshold
    3. d_index(t+window) < jump (because the timeseries start at the time when the running window ends)
 4. remove events that are too close in time to the event identified prior (within window+1 days) 

In [13]:
def read_in_ERA5_index(index, diri, basin, scale):
    # ds = xr.open_dataset(f'{diri}/ESI7_test.nc{index}{scale}_{basin}.nc')
    ds = xr.open_dataset(f'{diri}/{index}-{scale}_{basin}.nc')
    da = ds[f'{index}-{scale}']
    return da

In [3]:
def first_order_deriv_rolling_sum(da, window):
    dda = da[0:-1].copy(deep=True)
    dda.values = pd.Series(np.diff(da)).rolling(window=window).sum()
    
    return dda

In [78]:
def compute_FDs(da, dda, window, start_threshold, end_threshold, jump):
    '''
    da is the index timeseries
    dda is the time derivate of da, with rolling window
    window is the rolling window of the dda
    start_threshold is below with value da should start at the beginning of the flash drought
    end_threshold is below which value da should end after #window days after beginning of flash drought
    jump is the minimal value of dda at #window days after beginning of flash drought
    '''
    flash_drought_dates = []
    for i in da.time.data:
        i_end = i+np.timedelta64(window,'D')
        if da.sel(time=i,method='nearest') > start_threshold and da.sel(time=i_end,method='nearest') < end_threshold and dda.sel(time=i_end,method='nearest') < jump:
            flash_drought_dates.append(i.astype('datetime64[D]'))
    df = pd.DataFrame(data={'FD_startdate':flash_drought_dates})

    # cluster events that are consecutive days
    df['event'] = df['FD_startdate'].diff().dt.days.ne(1).cumsum()
    # group these events in a new df
    df_temp = df.groupby('event')['FD_startdate'].agg(['min', 'max'])

    # take individual events that are window+7 days apart from each other and cluster them
    df_temp['indiv_event'] = df_temp['min'].diff().dt.days.ge(window+7).cumsum()

    # group these events in a new df
    df_new = df_temp.groupby('indiv_event')['min'].agg(['min'])
    df_new['FD_startdate'] = df_new['min']
    df_new = df_new.drop(columns=['min'])
    df_new['year'] = df_new.FD_startdate.map(lambda x: x.year)
    df_new['month'] = df_new.FD_startdate.map(lambda x: x.month)
    df_new['day'] = df_new.FD_startdate.map(lambda x: x.day)
    
    return df_new 

In [79]:
df_test = compute_FDs(da, dda, window, start_threshold, end_threshold, jump)

In [5]:
def save_df_tocsv(df, diro, basin, index, scale, window, jump):
    filo = f'{basin}_{index}{scale}_dwindow{window}_jump{jump}.csv'
    df.to_csv(diro+filo)  

In [ ]:
def main():
    diri = '/scratch/nklm/Px_flashdroughts/indices_ERA5/'
    diro = '/perm/nklm/Px_flashdroughts/ERA5_FD_events/'
    basin = 'Rhine'
    start_threshold = 0
    end_threshold = -1
    for index in ['SPI','SPEI','ESI','SMI']:
        for scale in [7,14,21,28]:
            for window in [7,14,21,28]:
                for jump in np.arange(-1.5,-5,0.5): 
                    da = read_in_ERA5_index(index, diri, basin, scale)
                    dda = first_order_deriv_rolling_sum(da, window)
                    df = compute_FDs(da, dda, window, start_threshold, end_threshold, jump)
                    save_df_tocsv(df, diro, basin, index, scale, window, jump)


In [18]:
diri = '/scratch/nklm/Px_flashdroughts/indices_ERA5/'
# diro = '/perm/nklm/Px_flashdroughts/ERA5_FD_events/'
basin = 'Rhine'
start_threshold = -3
end_threshold = -1
index = 'ESI'
scale = 14
window = 21
jump = -2


In [19]:
da = read_in_ERA5_index(index, diri, basin, scale)
dda = first_order_deriv_rolling_sum(da, window)
df = compute_FDs(da, dda, window, start_threshold, end_threshold, jump)

In [44]:
def find_independend_FDs(df,window):
    # cluster events that are consecutive days
    df['event'] = df['FD_startdate'].diff().dt.days.ne(1).cumsum()
    # group these events in a new df
    df_temp = df.groupby('event')['FD_startdate'].agg(['min', 'max'])

    # take individual events that are window+7 days apart from each other
    df_temp['indiv_event'] = df_temp['min'].diff().dt.days.ge(window+7).cumsum()

    # write out these events to a clean df

    return df_temp

In [70]:
df_temp = find_independend_FDs(df,window)

In [71]:
df_temp

,min,max,indiv_event
event,,,
1,1950-11-26,1950-12-06,0
2,1951-12-02,1951-12-04,1
3,1952-01-12,1952-01-12,2
4,1952-03-24,1952-03-28,3
5,1952-11-01,1952-11-09,4
...,...,...,...
116,2019-03-09,2019-03-13,102
117,2020-03-07,2020-03-23,103
118,2020-10-27,2020-10-27,104


In [76]:
df_new = df_temp.groupby('indiv_event')['min'].agg(['min'])
df_new['FD_startdate'] = df_new['min']
df_new = df_new.drop(columns=['min'])
df_new['year'] = df_new.FD_startdate.map(lambda x: x.year)
df_new['month'] = df_new.FD_startdate.map(lambda x: x.month)
df_new['day'] = df_new.FD_startdate.map(lambda x: x.day)

In [77]:
df_new

,FD_startdate
indiv_event,
0,1950-11-26
1,1951-12-02
2,1952-01-12
3,1952-03-24
4,1952-11-01
...,...
102,2019-03-09
103,2020-03-07
104,2020-10-27


In [80]:
df_test

,FD_startdate,year,month,day
indiv_event,,,,
0,1950-11-26,1950,11,26
1,1951-12-02,1951,12,2
2,1952-01-12,1952,1,12
3,1952-03-24,1952,3,24
4,1952-11-01,1952,11,1
...,...,...,...,...
102,2019-03-09,2019,3,9
103,2020-03-07,2020,3,7
104,2020-10-27,2020,10,27


In [74]:
df_new['FD_startdate'] = df_new['min']

In [63]:
df_new['FD_startdate'] = df_new['min']
df_new.drop(['min'], axis=1)
df_new['year'] = df_new.FD_startdate.map(lambda x: x.year)
df_new['month'] = df_new.FD_startdate.map(lambda x: x.month)
df_new['day'] = df_new.FD_startdate.map(lambda x: x.day)

In [69]:
df_new.drop(columns=['min'])


,FD_startdate,year,month,day
indiv_event,,,,
0,1950-11-26,1950,11,26
1,1951-12-02,1951,12,2
2,1952-01-12,1952,1,12
3,1952-03-24,1952,3,24
4,1952-11-01,1952,11,1
...,...,...,...,...
102,2019-03-09,2019,3,9
103,2020-03-07,2020,3,7
104,2020-10-27,2020,10,27


In [22]:
df['diff'] = df['FD_startdate'].diff().dt.days.ne(1).cumsum()

In [25]:
df[0:60]

,FD_startdate,year,month,day,diff
0,1950-11-26,1950,11,26,1
1,1950-11-27,1950,11,27,1
2,1950-11-28,1950,11,28,1
3,1950-11-29,1950,11,29,1
4,1950-11-30,1950,11,30,1
5,1950-12-01,1950,12,1,1
6,1950-12-02,1950,12,2,1
7,1950-12-03,1950,12,3,1
8,1950-12-04,1950,12,4,1
9,1950-12-05,1950,12,5,1


In [27]:
result = df.groupby('diff')['FD_startdate'].agg(['min', 'max'])

In [28]:
result

,min,max
diff,,
1,1950-11-26,1950-12-06
2,1951-12-02,1951-12-04
3,1952-01-12,1952-01-12
4,1952-03-24,1952-03-28
5,1952-11-01,1952-11-09
...,...,...
116,2019-03-09,2019-03-13
117,2020-03-07,2020-03-23
118,2020-10-27,2020-10-27


In [38]:

result['diff'] = result['min'].diff().dt.days.ge(window+7).cumsum()

In [37]:
result

,min,max,diff
diff,,,
1,1950-11-26,1950-12-06,0
2,1951-12-02,1951-12-04,1
3,1952-01-12,1952-01-12,2
4,1952-03-24,1952-03-28,3
5,1952-11-01,1952-11-09,4
...,...,...,...
116,2019-03-09,2019-03-13,102
117,2020-03-07,2020-03-23,103
118,2020-10-27,2020-10-27,104


In [32]:
# array = [da.T.values,dda.T.values]
darray = np.array([da]).T
ddarray = np.array([dda]).T

In [60]:
ddarray 

array([       nan,        nan,        nan, ..., 2.87333343, 2.75688537,
              nan])

In [44]:
df = pd.DataFrame(darray, columns = ['da'])
# df = pd.DataFrame((darray,ddarray), columns = ['da','dda'])

In [59]:
ddarray = np.append(ddarray,np.nan)

In [52]:
ddarray.type

AttributeError: 'numpy.ndarray' object has no attribute 'type'

In [50]:
ddarray[4379] = np.nan

IndexError: index 4379 is out of bounds for axis 0 with size 4379

In [61]:
df['dda'] = ddarray.T

In [62]:
df

,da,dda
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
4375,1.554183,2.839243
4376,1.583280,2.853598
4377,1.679824,2.873333
4378,1.728841,2.756885


In [ ]:

FD = df.loc[df['da'] > start_threshold and ]

In [63]:
flash_drought_dates = []
for i in da[0:365].time.data:
    i_end = i+np.timedelta64(window,'D')
    if da.sel(time=i,method='nearest') > start_threshold and da.sel(time=i_end,method='nearest') < end_threshold and dda.sel(time=i_end,method='nearest') < jump:
        
        flash_drought_dates.append(i.astype('datetime64[D]'))
df_time = pd.DataFrame(data={'FD_startdate':flash_drought_dates})

In [67]:
df_time

,FD_startdate,year,month,day
0,1980-02-06,1980,2,6
1,1980-02-07,1980,2,7
2,1980-02-08,1980,2,8
3,1980-02-09,1980,2,9
4,1980-02-10,1980,2,10
5,1980-02-11,1980,2,11
6,1980-02-12,1980,2,12
7,1980-02-13,1980,2,13
8,1980-02-14,1980,2,14
9,1980-02-15,1980,2,15


In [66]:
df['year'] = df.FD_startdate.map(lambda x: x.year)
df['month'] = df.FD_startdate.map(lambda x: x.month)
df['day'] = df.FD_startdate.map(lambda x: x.day)

In [ ]:
if __name__ == '__main__':
    sys.exit(main())

In [8]:
df

,FD_startdate
0,1987-12-10


In [9]:
save_df_tocsv(df, diro, basin, index, scale, window, jump)